In [1]:
import os 
import re
import numpy as np
from rdkit import Chem as Ch
from rdkit.Chem import rdMolDescriptors
from collections import Counter
import itertools
from scipy.stats import multinomial

#Function to get average (exact) mw from ms1 files
    
def calculate_isotope_probabilities(element, count, isotopic_probabilities):
    probabilities = isotopic_probabilities.get(element)
    count_combinations = itertools.product(range(count + 1), repeat=len(probabilities))
    probabilities_dict = {}
    for counts in count_combinations:
        mult_dist = multinomial(count, probabilities)
        probability = mult_dist.pmf(counts)
        if probability >= 1e-10:
            probabilities_dict.update({counts:probability})
    return(probabilities_dict)

def get_element_counts(smiles):
    mol = Ch.MolFromSmiles(smiles)
    formula = Ch.rdMolDescriptors.CalcMolFormula(mol)
    element_counts = Counter()       
    # Use regular expression to extract elements and counts
    pattern = re.compile(r'([A-Z][a-z]*)([0-9]*)')
    matches = pattern.findall(formula)
    for match in matches:
        element = match[0]
        count = int(match[1]) if match[1] else 1
        element_counts[element] += count            
    return(dict(element_counts))

def get_intensities(smiles, isotopic_probabilities):
    #Get elements
    element_counts = get_element_counts(smiles)
    isotope_probabilities_list = []
    #Get the probabilitiy for every element and its count in order
    for element , count in element_counts.items():
        probabilities = calculate_isotope_probabilities(element, count, isotopic_probabilities)
        isotope_probabilities_list.append(probabilities)
    #Isocombis is list of list for all the isotope combinations
    isocombis = []
    for combi in isotope_probabilities_list:
        isocombis.append(list(combi.keys()))
    combinations = list(itertools.product(*isocombis))  
    elements = list(element_counts.keys())   
    iso_counter = Counter()
    #Get the masses associated with each combinition and sum them
    for combination in combinations:
        iso_mass = 0
        probabilities = []
        for combi in combination:
            combi_index = combination.index(combi)
            iso_mass += np.dot(combi,np.arange(0,len(combi)))
            probabilities.append(isotope_probabilities_list[combi_index].get(combi))
        if iso_mass <= 10:
            iso_counter[iso_mass] += np.prod(probabilities)
    output = []
    for i in range(len(iso_counter.keys())):
        output.append(iso_counter[i])
    output = list(output)
    #output = list(output/np.max(output))
    return(output)

isotopic_probabilities = {"H": [0.999855, 0.000145],
                          "B": [0.1978, 0.8022],
                          "C": [0.9889, 0.0111],
                          "N": [0.9963, 0.0037],
                          "O": [0.99757, 0.00038, 0.00205],
                          "F": [1.0],
                          "Mg": [0.79, 0.1, 0.11],
                          "Na": [1.0],
                          "Si": [0.9223, 0.0467, 0.031],
                          "P": [1.0],
                          "S": [0.95, 0.0075, 0.0425, 0, 0.0002],
                          "Cl": [0.7577, 0, 0.2423],
                          "K": [0.9326, 0.00012, 0.0673],
                          "Fe": [0.0584, 0, 0.9166, 0.0219, 0.0028],
                          "Co": [1.0],
                          "As": [1.0],
                          "Se": [0.0089, 0, 0.0937, 0.0763, 0.2377, 0, 0.4961, 0, 0.0873],
                          "Br": [0.5069, 0, 0.4931],
                          "I": [1.0]}

In [2]:
test1 = get_intensities("CCC1CCCNC(=O)C(CCCC(CCC1OC2C(C(C(C(O2)C)O)N)O)C)CC", isotopic_probabilities)

In [3]:
test1

[0.7451732430409226,
 0.2126674574528877,
 0.036866032102844726,
 0.00475298370302754,
 0.0004936259859912291,
 4.316686936485353e-05,
 3.260954916105499e-06,
 2.1629570955316934e-07,
 1.2666967158228969e-08,
 6.485759089462766e-10,
 2.9007332700139816e-11]